In [1]:
mp_pdf = True
import sys, os
if mp_pdf:
    extension = "pdf"
    import matplotlib
    matplotlib.use('pgf')
    pgf_with_latex = {
        "text.usetex": True,            # use LaTeX to write all text
        "pgf.rcfonts": False,           # Ignore Matplotlibrc
        "text.latex.unicode": True,
        "pgf.preamble": [
            #r"\usepackage[utf8x]{inputenc}",
            r"\usepackage{xcolor}"
        ],
        "pgf.texsystem" : "xelatex",
        "figure.figsize": [12,7]
    }
    matplotlib.rcParams.update(pgf_with_latex)
else:
    extension = "png"
    
run_id=1771
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from adjustText import adjust_text, get_renderer, get_bboxes

/home/max/.local/lib/python3.6/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [2]:
mdf = pd.read_csv('../tables/newness_representation.csv')
mdf.head()

mdf[mdf['primary_wg']==3]

,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,lrep,x_q,y_q
0,0.038335,4.114427,0.001480,3,218.514466,0.005529,"{coal, combustion, technology}",-0.004049,0.004049,0.267688,0.464458,4.588235,-1.317933,3,0
5,0.061647,5.939852,0.002137,3,217.785654,0.005510,"{soc, stock, soil}",-0.003374,0.003374,0.387745,0.428359,4.630435,-0.947407,4,0
9,0.190830,45.156800,0.016243,3,474.334290,0.012001,"{country, develop, international}",0.004242,0.004242,1.353438,0.435626,4.595745,0.302648,3,4
10,0.043010,7.954098,0.002861,3,439.285942,0.011114,"{cement, material, concrete}",-0.008253,0.008253,0.257421,0.392193,4.627907,-1.357044,4,0
23,0.149062,51.679730,0.018589,3,677.491379,0.017141,"{emission, ghg, reduction}",0.001448,0.001448,1.084467,0.447344,4.551020,0.081089,2,3
34,0.080107,18.479361,0.006647,3,526.000865,0.013308,"{energy, consumption, demand}",-0.006661,0.006661,0.499460,0.402016,4.651163,-0.694229,5,1
38,0.048817,6.561349,0.002360,3,306.572090,0.007757,"{power, generation, electricity}",-0.005396,0.005396,0.304271,0.415128,4.659091,-1.189837,5,0
39,0.022719,3.186839,0.001146,3,286.382796,0.007246,"{hydrogen, catalyst, cell}",-0.006099,0.006099,0.158202,0.460057,4.632653,-1.843880,4,0
44,0.036333,4.191450,0.001508,3,260.707918,0.006596,"{waste, landfill, treatment}",-0.005089,0.005089,0.228565,0.419463,4.586957,-1.475934,3,0
56,0.052956,6.039647,0.002172,3,289.901652,0.007335,"{vehicle, electric, transport}",-0.005162,0.005162,0.296183,0.380297,4.634146,-1.216776,4,0


In [3]:
topic_dis_df = pd.read_csv('../tables/topic_discipline.csv')
topic_dis_df = topic_dis_df.rename(columns={"dis_share":"dis_score"})

topic_dis_df['dis_total'] = topic_dis_df.groupby('title')['dis_score'].transform('sum')

topic_dis_df['dis_share'] = topic_dis_df['dis_score'] / topic_dis_df['dis_total']

topic_dis_df.head()

,dis_score,doctopic__doc__wc__oecd,primary_wg,title,dis_total,dis_share
0,40.160252,Agricultural Sciences,2,"{adaptation, vulnerability, farmer}",422.856596,0.094974
1,13.444909,Agricultural Sciences,2,"{adsorption, capacity, adsorbent}",316.221218,0.042517
2,5.938442,Agricultural Sciences,1,"{aerosol, forcing, radiative}",253.744444,0.023403
3,15.398295,Agricultural Sciences,1,"{arctic, warm, tundra}",386.863329,0.039803
4,68.393232,Agricultural Sciences,2,"{area, region, distribution}",720.605653,0.094911


In [4]:
topic_dis_df[topic_dis_df['primary_wg']==3]

,dis_score,doctopic__doc__wc__oecd,primary_wg,title,dis_total,dis_share
5,73.145604,Agricultural Sciences,3,"{biochar, application, amendment}",228.700732,0.319831
9,52.113985,Agricultural Sciences,3,"{carbon, low, sink}",502.786107,0.103650
10,26.580532,Agricultural Sciences,3,"{cement, material, concrete}",546.824036,0.048609
17,5.508942,Agricultural Sciences,3,"{coal, combustion, technology}",271.277967,0.020307
20,29.577818,Agricultural Sciences,3,"{cost, price, technology}",548.914052,0.053884
21,35.626645,Agricultural Sciences,3,"{country, develop, international}",570.626549,0.062434
28,88.898279,Agricultural Sciences,3,"{emission, ghg, reduction}",813.491561,0.109280
29,24.679252,Agricultural Sciences,3,"{energy, consumption, demand}",670.737482,0.036794
36,19.735911,Agricultural Sciences,3,"{fuel, fossil, engine}",386.082813,0.051118
44,9.021178,Agricultural Sciences,3,"{hydrogen, catalyst, cell}",341.706635,0.026400


In [5]:
topic_dis_wide = topic_dis_df.pivot_table(
    index=["title"],columns="doctopic__doc__wc__oecd",values="dis_share"
).reset_index()

topic_dis_wide.head()

doctopic__doc__wc__oecd,title,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences
0,"{adaptation, vulnerability, farmer}",0.094974,0.117893,0.006440,0.017409,0.530568,0.230353
1,"{adsorption, capacity, adsorbent}",0.042517,0.372730,0.001865,0.004678,0.549834,0.026996
2,"{aerosol, forcing, radiative}",0.023403,0.164013,0.002101,0.004009,0.785627,0.020331
3,"{arctic, warm, tundra}",0.039803,0.077289,0.004430,0.008271,0.826665,0.042920
4,"{area, region, distribution}",0.094911,0.159346,0.004530,0.014595,0.660921,0.064586


In [6]:
df = pd.concat([mdf.sort_values('title').reset_index(drop=True),topic_dis_wide.sort_values('title').reset_index(drop=True)],axis=1)
df = pd.merge(mdf, topic_dis_wide)
df.head()

,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,representation,...,year_av,lrep,x_q,y_q,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences
0,0.038335,4.114427,0.001480,3,218.514466,0.005529,"{coal, combustion, technology}",-0.004049,0.004049,0.267688,...,4.588235,-1.317933,3,0,0.020307,0.646801,0.001659,0.003217,0.259643,0.066686
1,0.150624,63.912225,0.022989,2,787.598419,0.019927,"{increase, concentration, effect}",0.003062,0.003062,1.153663,...,4.431373,0.142942,1,4,0.146045,0.167737,0.001887,0.012555,0.628161,0.042587
2,0.276211,39.111254,0.014068,1,231.722640,0.005863,"{cloud, feedback, radiative}",0.008205,0.008205,2.399568,...,4.315789,0.875289,0,5,0.025529,0.173084,0.001379,0.002189,0.776429,0.020964
3,0.127695,24.108271,0.008672,2,354.217701,0.008962,"{vegetation, ndvi, index}",-0.000290,0.000290,0.967600,...,4.423077,-0.032937,1,3,0.087178,0.127372,0.002122,0.004187,0.749989,0.028500
4,0.157020,23.672996,0.008515,1,278.090374,0.007036,"{solar, radiation, irradiance}",0.001479,0.001479,1.210228,...,4.423077,0.190809,1,4,0.032884,0.403613,0.004865,0.004484,0.512019,0.040386


In [7]:
pdf = df[df['primary_wg']==3]

fig, ax = plt.subplots(figsize=(8,5))

ax.scatter(
    pdf['Social Sciences'], 
    pdf['lrep'],
    c="#8da0cb",
    linewidths=1,
    edgecolors="grey"
)
texts = []
for i, row in pdf.iterrows():
    stext = row['title'].split(',')[0].replace('{','')
    texts.append(ax.text(row['Social Sciences'], row['lrep'], stext))

ax.axhline(0,c="grey")
ax.set_xlabel("Social science proportion")
ax.set_ylabel("IPCC representation (log)")

adjust_text(texts)

fig.patch.set_facecolor('#f0f0f0') 

#fig.tight_layout()

plt.savefig(
    f'../plots/run_{run_id}_wg3_socsci.{extension}',
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)
plt.show()

/home/max/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [8]:
fig, axs = plt.subplots(3,figsize=(8,12))
colors=["#66c2a5","#fc8d62","#8da0cb"]
for wg in [1,2,3]:
    
    ax = axs[wg-1]

    pdf = df[df['primary_wg']==wg]

    ax.scatter(
        pdf['Social Sciences'], 
        pdf['lrep'],
        c=colors[wg-1],
        linewidths=1,
        edgecolors="grey"
    )
    texts = []
    for i, row in pdf.iterrows():
        stext = row['title'].split(',')[0].replace('{','')
        texts.append(ax.text(row['Social Sciences'], row['lrep'], stext))

    ax.axhline(0,c="grey")
    ax.set_xlabel("Social science proportion")
    ax.set_ylabel("IPCC representation (log)")

    #adjust_text(texts)
    
fig.tight_layout()

plt.savefig(
    f'../plots/run_{run_id}_wgs_socsci.{extension}',
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
